In [1]:
import os
import csv
from os import path
import pandas as pd
import locale
import numpy as np
import math

#pd.set_option('display.max_rows', 500)
pd.reset_option('display.max_rows')

In [2]:
def get_full_file_path(file_path_list, file_name):
    """
    get_full_file_path returns os-specific path for a given filename and path
    :param file_path_list: file path
    :param file_name: file name
    :return: file path string
    """
    path_list = file_path_list + [file_name]
    return os.path.join(*path_list)  # splat * to unpack list arg

def import_csv(filename, import_path=['./']):
    """
    ingest_csv tries to import and read a csv file
    :param filename: string containing filename
    :param import_path: string for specified input folder
    :return: list of data
    """
    full_filepath = get_full_file_path(import_path, filename)  # assume files are in specific directory
    try:
        if path.isfile(full_filepath):
            # copied from https://automatetheboringstuff.com/chapter14/
            file = open(full_filepath, encoding="utf-8")
            reader = csv.reader(file)
            data = list(reader)
            file.close()
            return data
        else:
            raise Exception(FileNotFoundError, full_filepath + ' is not found')
    except Exception as instance:
        return str(instance)
    
def get_dataframe_from_list(data, headers=None):
    """
    get_dataframe_from_list translates raw data to pandas dataframe
    :param data: raw data to convert
    :param headers: list of strings for column titles
    :return: dataframe
    """
    if headers is None:
        headers = data.pop(0)
    return pd.DataFrame(data, columns=headers)

def export_dataframe_to_csv(dataframe, filename, export_path="output"):
    """
    export_dataframe_to_csv exports a pandas dataframe to the local directory as a csv
    :param dataframe: pandas dataframe to be written to file
    :param filename: string containing filename
    :param export_path: string for specified output folder
    :return: boolean
    """
    export_path_list = [export_path]
    full_filepath = get_full_file_path(export_path_list, filename)  # assume files are in specific directory
    dataframe.to_csv(path_or_buf=full_filepath)
    return True

In [3]:
column_id = "id"
column_name = 'name'
column_top_ten = 'top_ten'
column_top_theme = 'top_theme'
column_bottom_five = 'bottom_five'
column_bottom_theme = 'bottom_theme'
column_top_ten_count = 'top_ten_count'
column_top_ten_percentage = 'top_ten_percentage'
column_bottom_five_count = 'bottom_five_count'
column_bottom_five_percentage = 'bottom_five_percentage'
column_theme_overall_count = 'theme_overall_count'
column_theme_participant_count = 'theme_participant_count'

In [4]:
# import and clean the data
raw_data = import_csv('data.csv')
raw_headers = raw_data.pop(0)
df = get_dataframe_from_list(raw_data, raw_headers)
headers = list(filter(lambda x: len(x) > 0, raw_headers)) # remove empty strings from headers
df.drop(df.columns[range(len(headers), len(raw_headers)-1)], axis =1, inplace=True) # crop dataframe to only desired data
df = df.set_axis([column_id,column_name,column_top_ten,column_top_theme, column_bottom_five, column_bottom_theme], axis='columns')
df

,id,name,top_ten,top_theme,bottom_five,bottom_theme
0,1,28,Achiever,Executing,Context,Strategic Thinking
1,2,28,Competition,Influencing,Restorative,Executing
2,3,28,Strategic,Strategic Thinking,Deliberate,Executing
3,4,28,Relator,Relationship Building,Empathy,Relationship Building
4,5,28,Learner,Strategic Thinking,Connectedness,Relationship Building
...,...,...,...,...,...,...
215,216,21,Intellection,Strategic Thinking,,-
216,217,21,Connectedness,Relationship Building,,-
217,218,21,Analytical,Strategic Thinking,,-
218,219,21,Relator,Relationship Building,,-


In [5]:
# create separate dataframe for top results
top_df = df[[column_id, column_name,column_top_ten, column_top_theme]].copy()
top_df.drop(top_df[top_df[column_top_ten].str.len().lt(2)].index, inplace=True) # remove empty rows
top_df

,id,name,top_ten,top_theme
0,1,28,Achiever,Executing
1,2,28,Competition,Influencing
2,3,28,Strategic,Strategic Thinking
3,4,28,Relator,Relationship Building
4,5,28,Learner,Strategic Thinking
...,...,...,...,...
215,216,21,Intellection,Strategic Thinking
216,217,21,Connectedness,Relationship Building
217,218,21,Analytical,Strategic Thinking
218,219,21,Relator,Relationship Building


In [6]:
# create separate dataframe for bottom results
bottom_df = df[[column_id, column_name,column_bottom_five, column_bottom_theme]].copy()
bottom_df.drop(bottom_df[bottom_df[column_bottom_five].str.len().lt(2)].index, inplace=True) # remove empty rows
bottom_df

,id,name,bottom_five,bottom_theme
0,1,28,Context,Strategic Thinking
1,2,28,Restorative,Executing
2,3,28,Deliberate,Executing
3,4,28,Empathy,Relationship Building
4,5,28,Connectedness,Relationship Building
...,...,...,...,...
210,211,21,Restorative,Executing
211,212,21,Competition,Influencing
212,213,21,Deliberate,Executing
213,214,21,Consistency,Executing


In [7]:
# total participants
total_participants = df[column_name].nunique()
total_participants

22

In [8]:
# total themes
# combine top and bottom themes to ensure that themes across dataset are not missed
total_themes = pd.concat([top_df[column_top_theme], bottom_df[column_bottom_theme]]).unique()
total_themes

array(['Executing', 'Influencing', 'Strategic Thinking',
       'Relationship Building'], dtype=object)

In [9]:
# 1) Would be great to figure out how we can visualize the worksheet called “Top 10_Bottom 5”

In [10]:
# 2) Which of the 34 strengths appear in people’s list of Top 10 strengths most frequently? 
top_strengths = top_df[column_top_ten].value_counts()
top_strengths = top_strengths.to_frame().set_axis([column_top_ten_count], axis='columns')
top_strengths

,top_ten_count
Relator,16
Individualization,14
Learner,14
Strategic,13
Arranger,13
Futuristic,11
Maximizer,10
Input,10
Connectedness,9
Activator,9


In [11]:
# 2a) What percentage of people have each of these ‘most common top-10 strengths’?
top_strengths[column_top_ten_percentage] = (top_strengths[column_top_ten_count] / total_participants) * 100
top_strengths


,top_ten_count,top_ten_percentage
Relator,16,72.727273
Individualization,14,63.636364
Learner,14,63.636364
Strategic,13,59.090909
Arranger,13,59.090909
Futuristic,11,50.000000
Maximizer,10,45.454545
Input,10,45.454545
Connectedness,9,40.909091
Activator,9,40.909091


In [12]:
# 3) Which of the 34 strengths appear in people’s list of Bottom 5 strengths most frequently?
bottom_strengths = bottom_df[column_bottom_five].value_counts()
bottom_strengths = bottom_strengths.to_frame().set_axis([column_bottom_five_count], axis='columns')
bottom_strengths

,bottom_five_count
Harmony,11
Consistency,10
Deliberate,10
Context,9
Restorative,8
Discipline,6
Competition,6
Adaptability,5
Significance,5
Empathy,5


In [13]:
# 3a) What percentage of people have each of these ‘most common bottom-5 strengths’?
bottom_strengths[column_bottom_five_percentage] = (bottom_strengths[column_bottom_five_count] / total_participants) * 100
bottom_strengths

,bottom_five_count,bottom_five_percentage
Harmony,11,50.000000
Consistency,10,45.454545
Deliberate,10,45.454545
Context,9,40.909091
Restorative,8,36.363636
Discipline,6,27.272727
Competition,6,27.272727
Adaptability,5,22.727273
Significance,5,22.727273
Empathy,5,22.727273


In [14]:
# 4) Which of the four Themes is the most common Theme represented in the Top-10 Strengths? Second most? Third? Fourth?
top_themes = top_df[column_top_theme].value_counts().to_frame().set_axis([column_theme_overall_count], axis='columns')
top_themes


,theme_overall_count
Strategic Thinking,70
Relationship Building,63
Executing,40
Influencing,37


In [15]:
# 4a) How many people have each theme represented?
top_themes_by_participant = top_df.groupby([column_top_theme, column_name]).size().reset_index()
top_themes[column_theme_participant_count] = top_themes_by_participant[column_top_theme].value_counts()
top_themes



,theme_overall_count,theme_participant_count
Strategic Thinking,70,21
Relationship Building,63,22
Executing,40,18
Influencing,37,17


In [16]:
# 5) Which of the four Themes is the most common Theme represented in the Bottom-5 Strengths? Second most? Third? Fourth?
bottom_themes = bottom_df[column_bottom_theme].value_counts().to_frame().set_axis([column_theme_overall_count], axis='columns')
bottom_themes

,theme_overall_count
Executing,35
Relationship Building,28
Influencing,20
Strategic Thinking,17


In [17]:
# 5a) How many people have each theme represented?
bottom_themes_by_participant = bottom_df.groupby([column_bottom_theme, column_name]).size().reset_index()
bottom_themes[column_theme_participant_count] = bottom_themes_by_participant[column_bottom_theme].value_counts()
bottom_themes

,theme_overall_count,theme_participant_count
Executing,35,18
Relationship Building,28,17
Influencing,20,13
Strategic Thinking,17,12


In [18]:
# 6) Is there a Strength that no one has or very few people have in their Top-10?
unique_top_strengths = top_strengths[top_strengths[column_top_ten_count] == 1]
unique_top_strengths = unique_top_strengths[[column_top_ten_count]].copy() # reduce dataframe just 1 column
unique_top_strengths

,top_ten_count
Consistency,1


In [19]:
# 7) Is there a Strength that no one has or very few people have in their Bottom-5?
unique_bottom_strengths = bottom_strengths[bottom_strengths[column_bottom_five_count] == 1]
unique_bottom_strengths = unique_bottom_strengths[[column_bottom_five_count]].copy() # reduce dataframe just 1 column
unique_bottom_strengths

,bottom_five_count
Individualization,1
Communication,1
Achiever,1
Analytical,1
Ideation,1
Intellection,1
Command,1


In [20]:
# 8) Is there a Strength that only one person has in their Top-10, and that same strength is in other people’s Bottom-5?
# I think Consistency is this one


In [21]:
# 9) Who are the people who have Strengths in their Top-10 that are represented among all four Themes?
top_participants_by_theme = top_df.groupby([column_name, column_top_theme]).size().reset_index().rename(columns={0:'count'})
participants_by_theme_count_dictionary = dict(top_themes_by_participant[column_name].value_counts())
for k, v in list(participants_by_theme_count_dictionary.items()):
    if v != 4:
        del participants_by_theme_count_dictionary[k]
    else: 
        participants_by_theme_count_dictionary[k] = [v]
participants_by_theme_count = pd.DataFrame.from_dict(participants_by_theme_count_dictionary, orient='index',columns=['theme_count'])
participants_by_theme_count.index.name = column_name
participants_by_theme_count


,theme_count
name,
11,4
24,4
18,4
27,4
26,4
32,4
19,4
12,4
14,4


In [22]:
# 9a) Are there people whose Top-10 Strengths are only among three Themes? Can we see which Themes?
top_participants_by_theme = top_df.groupby([column_name, column_top_theme]).size().reset_index().rename(columns={0:'count'})
participants_by_theme_count_dictionary = dict(top_themes_by_participant[column_name].value_counts())
for k, v in list(participants_by_theme_count_dictionary.items()):
    if v != 3:
        del participants_by_theme_count_dictionary[k]
    else: 
        participants_by_theme_count_dictionary[k] = [v]
participants_by_theme_count = pd.DataFrame.from_dict(participants_by_theme_count_dictionary, orient='index',columns=['theme_count'])
participants_by_theme_count.index.name = column_name
participants_by_theme_count

,theme_count
name,
29,3
31,3
16,3
23,3
17,3
13,3


In [23]:
# 9b) What about people who only have strengths in two themes? Or just in one theme?
top_participants_by_theme = top_df.groupby([column_name, column_top_theme]).size().reset_index().rename(columns={0:'count'})
participants_by_theme_count_dictionary = dict(top_themes_by_participant[column_name].value_counts())
for k, v in list(participants_by_theme_count_dictionary.items()):
    if v != 2:
        del participants_by_theme_count_dictionary[k]
    else: 
        participants_by_theme_count_dictionary[k] = [v]
participants_by_theme_count = pd.DataFrame.from_dict(participants_by_theme_count_dictionary, orient='index',columns=['theme_count'])
participants_by_theme_count.index.name = column_name
participants_by_theme_count

,theme_count
name,
21,2
15,2


In [24]:
# 9c) What about people who only have strengths in one theme?
top_participants_by_theme = top_df.groupby([column_name, column_top_theme]).size().reset_index().rename(columns={0:'count'})
participants_by_theme_count_dictionary = dict(top_themes_by_participant[column_name].value_counts())
for k, v in list(participants_by_theme_count_dictionary.items()):
    if v != 1:
        del participants_by_theme_count_dictionary[k]
    else: 
        participants_by_theme_count_dictionary[k] = [v]
participants_by_theme_count = pd.DataFrame.from_dict(participants_by_theme_count_dictionary, orient='index',columns=['theme_count'])
participants_by_theme_count.index.name = column_name
participants_by_theme_count

,theme_count
name,


In [25]:
# 9d) all participant top theme counts
top_participants_by_theme = top_df.groupby([column_name, column_top_theme]).size().reset_index().rename(columns={0:'count'})
participants_by_theme_count_dictionary = dict(top_themes_by_participant[column_name].value_counts())
participants_by_theme_count = pd.DataFrame.from_dict(participants_by_theme_count_dictionary, orient='index',columns=['theme_count'])
participants_by_theme_count.index.name = column_name
participants_by_theme_count

,theme_count
name,
11,4
24,4
18,4
27,4
26,4
32,4
19,4
12,4
14,4


In [26]:
# 9e) all participant bottom theme counts
bottom_participants_by_theme = bottom_df.groupby([column_name, column_bottom_theme]).size().reset_index().rename(columns={0:'count'})
bottom_participants_by_theme_count_dictionary = dict(bottom_participants_by_theme[column_name].value_counts())
bottom_participants_by_theme_count = pd.DataFrame.from_dict(bottom_participants_by_theme_count_dictionary, orient='index',columns=['theme_count'])
bottom_participants_by_theme_count.index.name = column_name
bottom_participants_by_theme_count


,theme_count
name,
18,4
30,4
22,4
25,4
12,3
13,3
17,3
19,3
31,3


In [27]:
# 9f) all participant top themes
top_participants_by_theme = top_df.groupby([column_name, column_top_theme]).size().reset_index().rename(columns={0:'count'})
top_participants_by_theme

,name,top_theme,count
0,11,Executing,1
1,11,Influencing,4
2,11,Relationship Building,2
3,11,Strategic Thinking,3
4,12,Executing,1
...,...,...,...
73,31,Strategic Thinking,4
74,32,Executing,1
75,32,Influencing,1
76,32,Relationship Building,3


In [28]:
# 9g) all participant bottom themes
bottom_participants_by_theme = bottom_df.groupby([column_name, column_bottom_theme]).size().reset_index().rename(columns={0:'count'})
bottom_participants_by_theme

,name,bottom_theme,count
0,11,Executing,2
1,11,Relationship Building,3
2,12,Executing,3
3,12,Influencing,1
4,12,Relationship Building,1
5,13,Executing,1
6,13,Relationship Building,1
7,13,Strategic Thinking,3
8,14,Executing,3
9,14,Relationship Building,1


In [29]:
# export data
export_dataframe_to_csv(top_df, 'top-ten-data.csv')
export_dataframe_to_csv(bottom_df, 'bottom-five-data.csv')
export_dataframe_to_csv(top_strengths, 'top-strengths.csv')
export_dataframe_to_csv(bottom_strengths, 'bottom-strengths.csv')
export_dataframe_to_csv(top_themes, 'top-themes.csv')
export_dataframe_to_csv(bottom_themes, 'bottom-themes.csv')
export_dataframe_to_csv(unique_top_strengths, 'unique-top-strengths.csv')
export_dataframe_to_csv(unique_bottom_strengths, 'unique-bottom-strengths.csv')
export_dataframe_to_csv(participants_by_theme_count, 'top-theme-count-by-participant.csv')
export_dataframe_to_csv(participants_by_theme_count, 'bottom-theme-count-by-participant.csv')
export_dataframe_to_csv(top_participants_by_theme, 'top-strength-count-by-theme-by-participant.csv')
export_dataframe_to_csv(bottom_participants_by_theme, 'bottom-strength-count-by-theme-by-participant.csv')

True